Credit:
This work is heavily borrowed from the following source:
Author: BARAN BINGÖL
Kaggle URL: https://www.kaggle.com/code/baranbingl/face-shape-detection-85-acc-on-test-set

In [ ]:
# !pip3 install numpy matplotlib pillow torch torchvision scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
os.environ["OMP_NUM_THREADS"] = "1"

import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torch.cuda.amp import GradScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"On device : {device}")

In [ ]:
# oval_training_path = os.path.join(DATASETS_PATH, 'training_set/Oval')
# example_image_name = random.choice(os.listdir(oval_training_path))
# example_image_path = os.path.join(oval_training_path, example_image_name)
# example_image = Image.open(example_image_path)

# plt.imshow(example_image)
# plt.title(f"Image Size: {example_image.size}")
# plt.axis('off')
# plt.show()

In [ ]:
NUM_EPOCHS = 25
BATCH_SIZE = 24
NUM_WORKERS = os.cpu_count() - 1

DATASETS_PATH = '../datasets'
TRAIN_PATH = os.path.join(DATASETS_PATH, './training_set')
TEST_PATH = os.path.join(DATASETS_PATH, './testing_set')

In [ ]:
train_transforms = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(degrees=10),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def safe_pil_loader(path):
    try:
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')
    except (OSError, SyntaxError):
        return Image.new('RGB', (224, 224))

train_dataset = datasets.ImageFolder(root=TRAIN_PATH, transform=train_transforms, loader=safe_pil_loader)
test_dataset = datasets.ImageFolder(root=TEST_PATH, transform=test_transforms, loader=safe_pil_loader)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [ ]:
torch.manual_seed(42)

model = torchvision.models.efficientnet_b4(weights='EfficientNet_B4_Weights.IMAGENET1K_V1').to(device)
num_classes = len(train_dataset.classes)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5, inplace=True),
    nn.Linear(model.classifier[1].in_features, num_classes)
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999), weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

In [ ]:
def train_model(model, train_loader, criterion, optimizer, device, num_epochs):
    train_losses = []
    train_accuracies = []

    if device.type == 'cuda':
        scaler = GradScaler()
        # print("Using Scaler")
    else:
        scaler = None
        # print("Not using Scaler")

    for epoch in range(num_epochs):
        model.train()
        epoch_train_losses = []
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            if scaler is not None:
                # print("Using Scaler")
                with torch.cuda.amp.autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                # print("Not using Scaler")
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            epoch_train_losses.append(loss.item())

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        avg_train_loss = np.mean(epoch_train_losses)
        train_losses.append(avg_train_loss)

        accuracy = 100 * correct / total
        train_accuracies.append(accuracy)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {accuracy:.2f}%")

    # Plotting training loss
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss', color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plotting training accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), train_accuracies, label='Training Accuracy', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
def test_model(model, test_loader, criterion, device):
    model.eval()
    test_losses = []
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_losses.append(loss.item())

            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_test_loss = np.mean(test_losses)
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='macro')
    recall = recall_score(all_labels, all_predictions, average='macro')
    f1 = f1_score(all_labels, all_predictions, average='macro')

    print(f"Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

In [ ]:
def save_model(model, save_path):
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(model.state_dict(), save_path)

In [ ]:
train_model(model, train_loader, criterion, optimizer, device, NUM_EPOCHS)

In [ ]:
test_model(model, test_loader, criterion, device)

In [ ]:
save_model(model, '../outputs/best_model_self.pth')

In [ ]:
# !pip3 uninstall -y numpy matplotlib pillow torch torchvision scikit-learn